# Generate model if it does not yet exist

In [4]:
# # In case you do not have a .onnx model exported, use this cell to export your DLC3.0 snapshot

# from dlclive import DLCLive
# import cv2
# import numpy as np
# from pathlib import Path
# import time

# from deeplabcut.pose_estimation_pytorch.config import read_config_as_dict
# from deeplabcut.pose_estimation_pytorch.models import PoseModel
# import torch
# import onnxruntime as ort

# device = "cuda" if torch.cuda.is_available() else "cpu"

# #Anna
# root = Path("/Users/annastuckert/Downloads/Hand-AnnaStuckert-2024-08-21/dlc-models-pytorch/iteration-0/HandAug21-trainset95shuffle101/train")
# model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
# weights_path = root / "snapshot-200.pt"

# model = PoseModel.build(model_cfg["model"])
# weights = torch.load(weights_path, map_location=device)
# model.load_state_dict(weights["model"])

# dummy_input = torch.zeros((1, 3, 224, 224))

# torch.onnx.export(
#     model,
#     dummy_input,
#     "/Users/annastuckert/Downloads/Hand-AnnaStuckert-2024-08-21/dlc-models-pytorch/iteration-0/HandAug21-trainset95shuffle101/train/resnet.onnx",
#     verbose=False,
#     input_names=["input"],
#     dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"}},
# )

# Run live video inference

# Before Adding stopping mechanism

In [2]:

import csv
import os

import colorcet as cc
import cv2
import h5py
import numpy as np
from PIL import ImageColor


def analyze_live_video(
    dlc_live,
    camera: float = 0,
    experiment_name: str = "Test",
    pcutoff=0.5,
    display_radius=5,
    resize=None,
    save_poses=False,
    save_dir="model_predictions",
    draw_keypoint_names=False,
    cmap="bmy",
):
    """
    Analyze a video to track keypoints using an imported DeepLabCut model, visualize keypoints on the video, and optionally save the keypoint data and the labelled video.

    Parameters:
    -----------
    dlc_live : DLCLive
        An instance of the DLCLive class.
    camera : float, deafult=0 (webcam)
        The camera to record the live video from
    experiment_name : str, default = "Test"
        Prefix to label generated pose and video files
    pcutoff : float, optional, default=0.5
        The probability cutoff value below which keypoints are not visualized.
    display_radius : int, optional, default=5
        The radius of the circles drawn to represent keypoints on the video frames.
    resize : tuple of int (width, height) or None, optional, default=None
        The size to which the frames should be resized. If None, the frames are not resized.
    save_poses : bool, optional, default=False
        Whether to save the detected poses to CSV and HDF5 files.
    save_dir : str, optional, default="model_predictions"
        The directory where the output video and pose data will be saved.
    draw_keypoint_names : bool, optional, default=False
        Whether to draw the names of the keypoints on the video frames.
    cmap : str, optional, default="bmy"
        The colormap from the colorcet library to use for keypoint visualization.

    Returns:
    --------
    poses : list of dict
        A list of dictionaries where each dictionary contains the frame number and the corresponding pose data.
    """
    # Ensure save directory exists
    os.makedirs(name=save_dir, exist_ok=True)

    # Load video
    cap = cv2.VideoCapture(camera)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return
    # Start empty dict to save poses to for each frame
    poses = []
    # Create variable indicate current frame. Later in the code +1 is added to frame_index
    frame_index = 0

    # Load the DLC model
    try:
        pose_model = dlc_live.load_model()
    except Exception as e:
        print(f"Error: Could not load DLC model. Details: {e}")
        return

    # Retrieve bodypart names and number of keypoints
    bodyparts = dlc_live.cfg["metadata"]["bodyparts"]
    num_keypoints = len(bodyparts)

    # Set colors and convert to RGB
    cmap_colors = getattr(cc, cmap)
    colors = [
        ImageColor.getrgb(color)
        for color in cmap_colors[:: int(len(cmap_colors) / num_keypoints)]
    ]

    # Define output video path
    output_video_path = os.path.join(save_dir, f"{experiment_name}_DLCLIVE_LABELLED.mp4")

    # Get video writer setup
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width, frame_height = (
        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
    )

    if resize:
        frame_width, frame_height = resize
    vwriter = cv2.VideoWriter(
        filename=output_video_path,
        fourcc=fourcc,
        fps=fps,
        frameSize=(frame_width, frame_height),
    )

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        try:
            pose = dlc_live.get_pose(frame, pose_model=pose_model)
        except Exception as e:
            print(f"Error analyzing frame {frame_index}: {e}")
            continue

        poses.append({"frame": frame_index, "pose": pose})

        # Visualize keypoints
        this_pose = pose["poses"][0][0]
        for j in range(this_pose.shape[0]):
            if this_pose[j, 2] > pcutoff:
                x, y = map(int, this_pose[j, :2])
                cv2.circle(
                    frame,
                    center=(x, y),
                    radius=display_radius,
                    color=colors[j],
                    thickness=-1,
                )

                if draw_keypoint_names:
                    cv2.putText(
                        frame,
                        text=bodyparts[j],
                        org=(x + 10, y),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.5,
                        color=colors[j],
                        thickness=1,
                        lineType=cv2.LINE_AA,
                    )

        if resize:
            frame = cv2.resize(src=frame, dsize=resize)

        vwriter.write(image=frame)
        frame_index += 1

    cap.release()
    vwriter.release()

    if save_poses:
        save_poses_to_files(experiment_name, save_dir, bodyparts, poses)

    return poses


def save_poses_to_files(experiment_name, save_dir, bodyparts, poses):
    """
    Save the keypoint poses detected in the video to CSV and HDF5 files.

    Parameters:
    -----------
    experiment_name : str, default = "Test"
        Prefix to label generated pose and video files
    save_dir : str
        The directory where the pose data files will be saved.
    bodyparts : list of str
        A list of body part names corresponding to the keypoints.
    poses : list of dict
        A list of dictionaries where each dictionary contains the frame number and the corresponding pose data.

    Returns:
    --------
    None
    """
    base_filename = os.path.splitext(os.path.basename(experiment_name))[0]
    csv_save_path = os.path.join(save_dir, f"{base_filename}_poses.csv")
    h5_save_path = os.path.join(save_dir, f"{base_filename}_poses.h5")

    # Save to CSV
    with open(csv_save_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        header = ["frame"] + [
            f"{bp}_{axis}" for bp in bodyparts for axis in ["x", "y", "confidence"]
        ]
        writer.writerow(header)
        for entry in poses:
            frame_num = entry["frame"]
            pose = entry["pose"]["poses"][0][0]
            row = [frame_num] + [item for kp in pose for item in kp]
            writer.writerow(row)

    # Save to HDF5
    with h5py.File(h5_save_path, "w") as hf:
        hf.create_dataset(name="frames", data=[entry["frame"] for entry in poses])
        for i, bp in enumerate(bodyparts):
            hf.create_dataset(
                name=f"{bp}_x",
                data=[entry["pose"]["poses"][0][0][i, 0].item() for entry in poses],
            )
            hf.create_dataset(
                name=f"{bp}_y",
                data=[entry["pose"]["poses"][0][0][i, 1].item() for entry in poses],
            )
            hf.create_dataset(
                name=f"{bp}_confidence",
                data=[entry["pose"]["poses"][0][0][i, 2].item() for entry in poses],
            )




In [4]:
from dlclive import DLCLive

dlc_live = DLCLive(
    path="/Users/annastuckert/Downloads/Hand-AnnaStuckert-2024-08-21/dlc-models-pytorch/iteration-0/HandAug21-trainset95shuffle101/train",
    model_type="onnx",
    device="cpu",
    display=True,
)
#short video
#video_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/1_20cms_0degUP_first_1s.avi'
#working_directory =
#providing output directory
poses = analyze_live_video(dlc_live=dlc_live, camera=0, save_poses=True, save_dir='output_directory', draw_keypoint_names=True)


Loading DLC 3.0.0rc2...


/Users/annastuckert/anaconda3/envs/deeplabcut3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-21 16:20:13.156 python[43079:2032191] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


ONNX inference took 3.740891933441162 sec
ONNX inference took 3.3362958431243896 sec
ONNX inference took 3.5694820880889893 sec


: 

# Run it until you press Q

In [1]:


import csv
import os
import time
import cv2
import h5py
import colorcet as cc
import numpy as np
from PIL import ImageColor


def analyze_live_video(
    dlc_live,
    camera: float = 0,
    experiment_name: str = "Test",
    pcutoff=0.5,
    display_radius=5,
    resize=None,
    save_poses=False,
    save_dir="model_predictions",
    draw_keypoint_names=False,
    cmap="bmy",
):
    """
    Analyze a video to track keypoints using an imported DeepLabCut model, visualize keypoints on the video, and optionally save the keypoint data and the labelled video.

    Parameters:
    -----------
    dlc_live : DLCLive
        An instance of the DLCLive class.
    camera : float, default=0 (webcam)
        The camera to record the live video from
    experiment_name : str, default = "Test"
        Prefix to label generated pose and video files
    pcutoff : float, optional, default=0.5
        The probability cutoff value below which keypoints are not visualized.
    display_radius : int, optional, default=5
        The radius of the circles drawn to represent keypoints on the video frames.
    resize : tuple of int (width, height) or None, optional, default=None
        The size to which the frames should be resized. If None, the frames are not resized.
    save_poses : bool, optional, default=False
        Whether to save the detected poses to CSV and HDF5 files.
    save_dir : str, optional, default="model_predictions"
        The directory where the output video and pose data will be saved.
    draw_keypoint_names : bool, optional, default=False
        Whether to draw the names of the keypoints on the video frames.
    cmap : str, optional, default="bmy"
        The colormap from the colorcet library to use for keypoint visualization.

    Returns:
    --------
    poses : list of dict
        A list of dictionaries where each dictionary contains the frame number and the corresponding pose data.
    """
    # Ensure save directory exists
    os.makedirs(name=save_dir, exist_ok=True)

    # Load video
    cap = cv2.VideoCapture(camera)
    if not cap.isOpened():
        print(f"Error: Could not open video source {camera}")
        return

    # Define output video path
    output_video_path = os.path.join(save_dir, f"{experiment_name}_DLCLIVE_LABELLED.mp4")

    # Get video writer setup
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width, frame_height = (
        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
    )

    if resize:
        frame_width, frame_height = resize

    vwriter = cv2.VideoWriter(
        filename=output_video_path,
        fourcc=fourcc,
        fps=fps,
        frameSize=(frame_width, frame_height),
    )

    # Load the DLC model
    try:
        pose_model = dlc_live.load_model()
    except Exception as e:
        print(f"Error: Could not load DLC model. Details: {e}")
        cap.release()
        return

    # Retrieve bodypart names and number of keypoints
    bodyparts = dlc_live.cfg["metadata"]["bodyparts"]
    num_keypoints = len(bodyparts)

    # Set colors and convert to RGB
    cmap_colors = getattr(cc, cmap)
    colors = [
        ImageColor.getrgb(color)
        for color in cmap_colors[:: int(len(cmap_colors) / num_keypoints)]
    ]

    poses = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()
        try:
            pose = dlc_live.get_pose(frame, pose_model=pose_model)
        except Exception as e:
            print(f"Error analyzing frame {len(poses)}: {e}")
            continue

        # Log inference time
        inference_time = time.time() - start_time
        print(f"Frame {len(poses)} processed in {inference_time:.2f} seconds.")

        poses.append({"frame": len(poses), "pose": pose})

        # Visualize keypoints
        this_pose = pose["poses"][0][0]
        for j in range(this_pose.shape[0]):
            if this_pose[j, 2] > pcutoff:
                x, y = map(int, this_pose[j, :2])
                cv2.circle(
                    frame,
                    center=(x, y),
                    radius=display_radius,
                    color=colors[j],
                    thickness=-1,
                )

                if draw_keypoint_names:
                    cv2.putText(
                        frame,
                        text=bodyparts[j],
                        org=(x + 10, y),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.5,
                        color=colors[j],
                        thickness=1,
                        lineType=cv2.LINE_AA,
                    )

        if resize:
            frame = cv2.resize(src=frame, dsize=resize)

        vwriter.write(image=frame)

        # Display the frame (optional)
        cv2.imshow('Live Inference', frame)

        # Check if 'q' key was pressed to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Video capture stopped by user.")
            break

    # Release resources properly
    cap.release()
    vwriter.release()
    cv2.destroyAllWindows()

    if save_poses:
        save_poses_to_files(experiment_name, save_dir, bodyparts, poses)

    print("Analysis done.")
    return poses


def save_poses_to_files(experiment_name, save_dir, bodyparts, poses):
    """
    Save the keypoint poses detected in the video to CSV and HDF5 files.

    Parameters:
    -----------
    experiment_name : str, default = "Test"
        Prefix to label generated pose and video files
    save_dir : str
        The directory where the pose data files will be saved.
    bodyparts : list of str
        A list of body part names corresponding to the keypoints.
    poses : list of dict
        A list of dictionaries where each dictionary contains the frame number and the corresponding pose data.

    Returns:
    --------
    None
    """
    base_filename = os.path.splitext(os.path.basename(experiment_name))[0]
    csv_save_path = os.path.join(save_dir, f"{base_filename}_poses.csv")
    h5_save_path = os.path.join(save_dir, f"{base_filename}_poses.h5")

    # Save to CSV
    with open(csv_save_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        header = ["frame"] + [
            f"{bp}_{axis}" for bp in bodyparts for axis in ["x", "y", "confidence"]
        ]
        writer.writerow(header)
        for entry in poses:
            frame_num = entry["frame"]
            pose = entry["pose"]["poses"][0][0]
            row = [frame_num] + [item for kp in pose for item in kp]
            writer.writerow(row)

    # Save to HDF5
    with h5py.File(h5_save_path, "w") as hf:
        hf.create_dataset(name="frames", data=[entry["frame"] for entry in poses])
        for i, bp in enumerate(bodyparts):
            hf.create_dataset(
                name=f"{bp}_x",
                data=[entry["pose"]["poses"][0][0][i, 0].item() for entry in poses],
            )
            hf.create_dataset(
                name=f"{bp}_y",
                data=[entry["pose"]["poses"][0][0][i, 1].item() for entry in poses],
            )
            hf.create_dataset(
                name=f"{bp}_confidence",
                data=[entry["pose"]["poses"][0][0][i, 2].item() for entry in poses],
            )



# Run it for 1 second

In [5]:
import csv
import os
import time
import cv2
import h5py
import colorcet as cc
import numpy as np
from PIL import ImageColor
import cProfile
import pstats
from dlclive import DLCLive


def analyze_live_video(
    dlc_live,
    camera: float = 0,
    experiment_name: str = "Test",
    pcutoff=0.5,
    display_radius=5,
    resize=None,
    save_poses=False,
    save_dir="model_predictions",
    draw_keypoint_names=False,
    cmap="bmy",
):
    """
    Analyze a video to track keypoints using an imported DeepLabCut model, visualize keypoints on the video, and optionally save the keypoint data and the labelled video.

    Parameters:
    -----------
    dlc_live : DLCLive
        An instance of the DLCLive class.
    camera : float, default=0 (webcam)
        The camera to record the live video from
    experiment_name : str, default = "Test"
        Prefix to label generated pose and video files
    pcutoff : float, optional, default=0.5
        The probability cutoff value below which keypoints are not visualized.
    display_radius : int, optional, default=5
        The radius of the circles drawn to represent keypoints on the video frames.
    resize : tuple of int (width, height) or None, optional, default=None
        The size to which the frames should be resized. If None, the frames are not resized.
    save_poses : bool, optional, default=False
        Whether to save the detected poses to CSV and HDF5 files.
    save_dir : str, optional, default="model_predictions"
        The directory where the output video and pose data will be saved.
    draw_keypoint_names : bool, optional, default=False
        Whether to draw the names of the keypoints on the video frames.
    cmap : str, optional, default="bmy"
        The colormap from the colorcet library to use for keypoint visualization.

    Returns:
    --------
    poses : list of dict
        A list of dictionaries where each dictionary contains the frame number and the corresponding pose data.
    """
    # Ensure save directory exists
    os.makedirs(name=save_dir, exist_ok=True)

    # Load video
    cap = cv2.VideoCapture(camera)
    if not cap.isOpened():
        print(f"Error: Could not open video source {camera}")
        return

    # Define output video path
    output_video_path = os.path.join(save_dir, f"{experiment_name}_DLCLIVE_LABELLED.mp4")

    # Get video writer setup
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width, frame_height = (
        int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
    )

    if resize:
        frame_width, frame_height = resize

    vwriter = cv2.VideoWriter(
        filename=output_video_path,
        fourcc=fourcc,
        fps=fps,
        frameSize=(frame_width, frame_height),
    )

    # Load the DLC model
    try:
        pose_model = dlc_live.load_model()
    except Exception as e:
        print(f"Error: Could not load DLC model. Details: {e}")
        cap.release()
        return

    # Retrieve bodypart names and number of keypoints
    bodyparts = dlc_live.cfg["metadata"]["bodyparts"]
    num_keypoints = len(bodyparts)

    # Set colors and convert to RGB
    cmap_colors = getattr(cc, cmap)
    colors = [
        ImageColor.getrgb(color)
        for color in cmap_colors[:: int(len(cmap_colors) / num_keypoints)]
    ]

    poses = []
    frames = []
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Capture the frame and store it
        frames.append(frame.copy())

        # Calculate elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:  # Stop capturing after 1 second
            print("Recording stopped after 1 second.")
            break

        # Display the frame (optional)
        cv2.imshow('Recording...', frame)

        # Check if 'q' key was pressed to stop recording early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Video capture stopped by user.")
            break

    # Release the video capture resources
    cap.release()
    cv2.destroyAllWindows()

    # Start profiling the processing of frames
    profiler = cProfile.Profile()
    profiler.enable()

    # Process the captured frames
    for i, frame in enumerate(frames):
        try:
            pose = dlc_live.get_pose(frame, pose_model=pose_model)
        except Exception as e:
            print(f"Error analyzing frame {i}: {e}")
            continue

        poses.append({"frame": i, "pose": pose})

        # Visualize keypoints
        this_pose = pose["poses"][0][0]
        for j in range(this_pose.shape[0]):
            if this_pose[j, 2] > pcutoff:
                x, y = map(int, this_pose[j, :2])
                cv2.circle(
                    frame,
                    center=(x, y),
                    radius=display_radius,
                    color=colors[j],
                    thickness=-1,
                )

                if draw_keypoint_names:
                    cv2.putText(
                        frame,
                        text=bodyparts[j],
                        org=(x + 10, y),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.5,
                        color=colors[j],
                        thickness=1,
                        lineType=cv2.LINE_AA,
                    )

        if resize:
            frame = cv2.resize(src=frame, dsize=resize)

        vwriter.write(image=frame)

    # Stop profiling
    profiler.disable()

    # Release the video writer resources
    vwriter.release()

    if save_poses:
        save_poses_to_files(experiment_name, save_dir, bodyparts, poses)

    print("Analysis done.")

    # Print profiling results
    profiler.print_stats(sort='cumtime')

    return poses


def save_poses_to_files(experiment_name, save_dir, bodyparts, poses):
    """
    Save the keypoint poses detected in the video to CSV and HDF5 files.

    Parameters:
    -----------
    experiment_name : str, default = "Test"
        Prefix to label generated pose and video files
    save_dir : str
        The directory where the pose data files will be saved.
    bodyparts : list of str
        A list of body part names corresponding to the keypoints.
    poses : list of dict
        A list of dictionaries where each dictionary contains the frame number and the corresponding pose data.

    Returns:
    --------
    None
    """
    base_filename = os.path.splitext(os.path.basename(experiment_name))[0]
    csv_save_path = os.path.join(save_dir, f"{base_filename}_poses.csv")
    h5_save_path = os.path.join(save_dir, f"{base_filename}_poses.h5")

    # Save to CSV
    with open(csv_save_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        header = ["frame"] + [
            f"{bp}_{axis}" for bp in bodyparts for axis in ["x", "y", "confidence"]
        ]
        writer.writerow(header)
        for entry in poses:
            frame_num = entry["frame"]
            pose = entry["pose"]["poses"][0][0]
            row = [frame_num] + [item for kp in pose for item in kp]
            writer.writerow(row)

    # Save to HDF5
    with h5py.File(h5_save_path, "w") as hf:
        hf.create_dataset(name="frames", data=[entry["frame"] for entry in poses])
        for i, bp in enumerate(bodyparts):
            hf.create_dataset(
                name=f"{bp}_x",
                data=[entry["pose"]["poses"][0][0][i, 0].item() for entry in poses],
            )
            hf.create_dataset(
                name=f"{bp}_y",
                data=[entry["pose"]["poses"][0][0][i, 1].item() for entry in poses],
            )
            hf.create_dataset(
                name=f"{bp}_confidence",
                data=[entry["pose"]["poses"][0][0][i, 2].item() for entry in poses],
            )



In [15]:
from dlclive import DLCLive

dlc_live = DLCLive(
    path="/Users/annastuckert/Downloads/Hand-AnnaStuckert-2024-08-21/dlc-models-pytorch/iteration-0/HandAug21-trainset95shuffle101/train",
    model_type="onnx",
    device="cpu",
    display=True,
)


poses = analyze_live_video(
    dlc_live=dlc_live,
    camera=0,
    save_poses=True,
    save_dir='output_directory',
    draw_keypoint_names=True)

#OBS actually takes longer to run on resized than original videos!
# resized_width = 320  # Desired width of the video
# resized_height = 240  # Desired height of the video

# poses = analyze_live_video(
#     dlc_live=dlc_live,
#     camera=0,
#     save_poses=True,
#     save_dir='output_directory',
#     draw_keypoint_names=True,
#     resize=(resized_width, resized_height)  # Resize parameter
# )


Recording stopped after 1 second.
ONNX inference took 2.6377220153808594 sec
ONNX inference took 3.4302010536193848 sec
ONNX inference took 3.005439043045044 sec
ONNX inference took 2.7167038917541504 sec
ONNX inference took 3.2396998405456543 sec
ONNX inference took 3.641049861907959 sec
ONNX inference took 2.3327910900115967 sec
ONNX inference took 1.8497438430786133 sec
ONNX inference took 1.8688640594482422 sec
ONNX inference took 2.0275509357452393 sec
ONNX inference took 2.119837999343872 sec
ONNX inference took 1.881242275238037 sec
ONNX inference took 3.470150947570801 sec
ONNX inference took 3.8359780311584473 sec
ONNX inference took 2.255268096923828 sec
ONNX inference took 3.6002070903778076 sec
ONNX inference took 3.729128837585449 sec
ONNX inference took 3.662476062774658 sec
ONNX inference took 3.599925994873047 sec
ONNX inference took 3.7375919818878174 sec
ONNX inference took 2.479177236557007 sec
ONNX inference took 2.3914618492126465 sec
ONNX inference took 2.04442811